# Features

In [1]:
# coding: utf-8


# # Import text file from the user 

from tkinter import *
from tkinter import filedialog
import os
import numpy as np
root = Tk()
my_filetypes = [('all files', '.*'), ('text files', '.txt')]
root.filename =  filedialog.askopenfilename(
                                    title="Please select a file:",
                                    filetypes=my_filetypes)
print ("File directory:",root.filename)
print("\n")
root.withdraw()
#text=open(root.filename, encoding="utf-8").read()
text=open(root.filename).read()
print(text)
print("\n")



# # Sentence segmentation

from nltk import sent_tokenize
sentences=(sent_tokenize(text))
print("Sentences:",sentences)
print("\n")
#print(len(sentences))


emptyarray= np.empty((len(sentences),1,3),dtype=object)
for s in range(len(sentences)):
    emptyarray[s][0][0] = sentences[s]
    emptyarray[s][0][1] = s


# # Tokenization, Stop word removal , Bi-grams, Tri-grams

import nltk
from string import punctuation
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
bi_token=[]
bi_token_length=[]
tri_token_length=[]
for u in range(len(sentences)):
    sent_split1=[w.lower() for w in sentences[u].split(" ")]
    sent_split=[w for w in sent_split1 if w not in stop_words and w not in punctuation and not w.isdigit()]
    
    bigrams_list = [bigram for bigram in nltk.bigrams(sent_split)]
    bi_token.append(bigrams_list)
    bi_token_length.append(len(bi_token[u]))
bi_tokens = [(int(o) / max(bi_token_length))*100 for o in bi_token_length]
print("bitokens feature vector:",(bi_token_length))
#print(max(bi_token_length))
#print(bi_token_length)
print("\n")


tri_token=[]
for u in range(len(sentences)):
    sent_split2=[w.lower() for w in sentences[u].split(" ")]
    sent_split3=[w for w in sent_split2 if w not in stop_words and w not in punctuation and not w.isdigit()]
    trigrams_list = [trigram for trigram in nltk.trigrams(sent_split3)]
    tri_token.append(trigrams_list)
    tri_token_length.append(len(tri_token[u]))
tri_tokens = [(int(m) / max(tri_token_length))*100 for m in tri_token_length]

print("tritokens feature vector:",tri_token_length)
print("\n")



# # Sentence Position Feature

import math
def position(l):
    return [index for index, value in enumerate(sentences)]

sent_position= (position(sentences))
num_sent=len(sent_position)
print("sentence position:",sent_position)
print("\n")
print("Total number of sentences:",num_sent)
print("\n")
#th= 0.2*num_sent
#minv=th*num_sent
#maxv=th*2*num_sent
position = []
position_rbm = []
sent_pos1_rbm = 1
sent_pos1 = 100
position.append(sent_pos1)
position_rbm.append(sent_pos1_rbm)
for x in range(1,num_sent-1):
    #s_p = (math.cos((sent_position[x]-minv)*((1/maxv)-minv)))*100
    #if s_p < 0:
     #   s_p = 0
    s_p= ((num_sent-x)/num_sent)*100
    position.append(s_p)
    s_p_rbm = (num_sent-x)/num_sent
    position_rbm.append(s_p_rbm)
    
sent_pos2 = 100
sent_pos2_rbm = 1
position.append(sent_pos2)
position_rbm.append(sent_pos2_rbm)
print("Sentence position feature vector:",position_rbm)
print("\n")



# # Converting Sentences to Vectors

def convertToVSM(sentences):
    vocabulary = []
    for sents in sentences:
        vocabulary.extend(sents)
    vocabulary = list(set(vocabulary))
    vectors = []
    for sents in sentences:
        vector = []
        for tokenss in vocabulary:
            vector.append(sents.count(tokenss))
        vectors.append(vector)
    return vectors
VSM=convertToVSM(sentences)
print("SentenceVectors:",VSM)
print("\n")




# # TF-ISF feature and Centroid Calculation

sentencelength=len(sentences)
def calcMeanTF_ISF(VSM, index):
    vocab_len = len(VSM[index])
    sentences_len = len(VSM)
    count = 0
    tfisf = 0
    for i in range(vocab_len):
        tf = VSM[index][i]
        if(tf>0):
            count += 1
            sent_freq = 0
            for j in range(sentences_len):
                if(VSM[j][i]>0): sent_freq += 1
            tfisf += (tf)*(1.0/sent_freq)
    if(count > 0):
        mean_tfisf = tfisf/count
    else:
        mean_tfisf = 0
    return tf, (1.0/sent_freq), mean_tfisf
tfvec=[]
isfvec=[]
tfisfvec=[]
tfisfvec_rbm=[]
for i in range(sentencelength):
    x,y,z=calcMeanTF_ISF(VSM,i)
    tfvec.append(x)
    isfvec.append(y)
    tfisfvec.append(z*100)
    tfisfvec_rbm.append(z)
#print("TF vector:",tfvec)
#print("\n")
#print("ISF vector:",isfvec)
#print("\n")
#tfisf1= [(int(p)*100) for p in tfisfvec]
print("TF-ISF vector:",tfisfvec_rbm)
print("\n")
maxtf_isf=max(tfisfvec_rbm)
centroid=[]
centroid.append(maxtf_isf)
print("Max TF-ISF:",centroid)
print("\n")
#for q in range(sentencelength):
centroid=(max(VSM))
print("Centroid:",centroid)
print("\n")



# # Cosine Similarity between Centroid and Sentences

from numpy import dot
from numpy.linalg import norm
cosine_similarity=[]
cosine_similarity_rbm=[]
for z in range(sentencelength):
    cos_simi = ((dot(centroid, VSM[z])/(norm(centroid)*norm(VSM[z])))*100)
    cosine_similarity.append(cos_simi)
    cos_simi_rbm = (dot(centroid, VSM[z])/(norm(centroid)*norm(VSM[z])))
    cosine_similarity_rbm.append(cos_simi_rbm)
print("Cosine Similarity Vector:",cosine_similarity_rbm)
print("\n")




# # Sentence length feature

sent_word=[]
for u in range(len(sentences)):
    sent_split1=[w.lower() for w in sentences[u].split(" ")]
    sent_split=[w for w in sent_split1 if w not in stop_words and w not in punctuation and not w.isdigit()]
    a=(len(sent_split))
    sent_word.append(a)
#print("Number of words in each sentence:",sent_word)
#print("\n")
#sent_leng=[]
#for x in range(len(sentences)):
 #   if sent_word[x] < 3:
  #      sent_leng.append(0)
  #  else:
   #     sent_leng.append(1)

##OR BY THIS METHOD: LENGTH OF SENTENCE/ LONGEST SENTENCE
longest_sent=max(sent_word)
sent_length=[]
sent_length_rbm=[]
for x in sent_word:
    sent_length.append((x/longest_sent)*100)
    sent_length_rbm.append(x/longest_sent)
#print(sent_length)

print("Sentence length feature vector:",sent_length_rbm)
print("\n")




# # Numeric token Feature

import re
num_word=[]
numeric_token=[]
numeric_token_rbm=[]
for u in range(len(sentences)):
    sent_split4=sentences[u].split(" ")
    e=re.findall("\d+",sentences[u])
    noofwords=(len(e))
    num_word.append(noofwords)
    numeric_token.append((num_word[u]/sent_word[u])*100)
    numeric_token_rbm.append(num_word[u]/sent_word[u])
#print("Numeric word count in each sentence:",num_word)
#print("\n")
print("Numeric token feature vector:",numeric_token_rbm)
print("\n")



# # Thematic words feature
from rake_nltk import Rake
r = Rake() # Uses stopwords for english from NLTK, and all puntuation characters.
keywords=[]
# If you want to provide your own set of stop words and punctuations to
# r = Rake(<list of stopwords>, <string of puntuations to ignore>)

for s in sentences:
    r.extract_keywords_from_text(s)
    key=list(r.get_ranked_phrases())
    keywords.append(key)
#print(keywords)
l_keywords=[]
for s in keywords:
    leng=len(s)
    l_keywords.append(leng)
#print(l_keywords)

total_keywords=sum(l_keywords)
#print(total_keywords)

thematic_number= []
thematic_number_rbm= []
for x in l_keywords:
    thematic_number.append((x/total_keywords)*100)
    thematic_number_rbm.append(x/total_keywords)
print("Thematic word feature", thematic_number_rbm)
print("\n")



# # proper noun feature
from nltk.tag import pos_tag
from collections import Counter
pncounts = []
pncounts_rbm = []
for sentence in sentences:
    tagged=nltk.pos_tag(nltk.word_tokenize(str(sentence)))
    counts = Counter(tag for word,tag in tagged if tag.startswith('NNP') or tag.startswith('NNPS'))
    f=sum(counts.values())
    pncounts.append(f)
    pncounts_rbm.append(f)
pnounscore=[(int(o) / int(p))*100 for o,p in zip(pncounts, sent_word)]
pnounscore_rbm=[int(o) / int(p) for o,p in zip(pncounts_rbm, sent_word)]
#print(pncounts)
print("Pronoun feature vector",pnounscore_rbm)
print("\n")


# # feature matrix1


featureMatrix = []
featureMatrix.append(position_rbm)
featureMatrix.append(bi_token_length)
featureMatrix.append(tri_token_length)
featureMatrix.append(tfisfvec_rbm)
featureMatrix.append(cosine_similarity_rbm)
featureMatrix.append(thematic_number_rbm)
featureMatrix.append(sent_length_rbm)
featureMatrix.append(numeric_token_rbm)
featureMatrix.append(pnounscore_rbm)



featureMat = np.zeros((len(sentences),9))
for i in range(9) :
    for j in range(len(sentences)):
        featureMat[j][i] = featureMatrix[i][j]

print("\n\n\nPrinting Feature Matrix : ")
print(featureMat)
print("\n\n\nPrinting Feature Matrix Normed : ")
#featureMat_normed = featureMat / featureMat.max(axis=0)
featureMat_normed = featureMat

print(featureMat_normed)
for i in range(len(sentences)):
    print(featureMat_normed[i])
#np.save('output_labels_10.npy',featureMat_normed)


File directory: C:/Users/shashi/Desktop/cricket.txt


Johannesburg: South Africa survived a nail-biting finish to edge out India by five wickets in a rain-curtailed fourth One-Day International (ODI) in Johannesburg on Saturday.

India, who had won the first three ODIs, now have a 3-1 lead in the six-match series.

Electing to bat first, India rode on opener Shikhar Dhawan's 13th ODI century to post 289/7 in their 50 overs.

Dhawan scored 109 runs off 105 balls, and shared a 158-run stand with skipper Virat Kohli who scored 75.

The match was interrupted by rain and inclement weather on more than one occasion which saw the target being reduced to 202 runs off 28 overs for South Africa.

The Indian bowlers dominated South African batsmen in the initial stages.

With their top four batsmen back in the pavilion by the 17th over with just 102 runs on the board, the Proteas seemed to have lost their way and headed for certain defeat.

When David Miller was bowled while trying to hit an inco

Cosine Similarity Vector: [0.92813827375529134, 0.90944060125288395, 0.93482142857142858, 0.88540489199381023, 0.96657579117525383, 0.92693401419230848, 0.96332812440456506, 0.94195393536668182, 0.94446058047314818, 0.93106183051738134, 1.0, 0.91123764531694729, 0.94322172882611732, 0.95218294487720245, 0.93697586063664007, 0.92488153764479875, 0.9338470753797754, 0.94362496298106424, 0.92549127003397014, 0.94869395256501088, 0.91980040850166012, 0.85809780077027231, 0.8107775860972275, 0.89538831693027909, 0.93953211133802861, 0.91820317180339039, 0.95190881282349427, 0.94395543581350538, 0.8988393618391558, 0.88975866421698813, 0.90984810242275538, 0.94445015791250697]


Sentence length feature vector: [0.8947368421052632, 0.42105263157894735, 0.7368421052631579, 0.631578947368421, 0.7368421052631579, 0.42105263157894735, 0.7894736842105263, 0.8421052631578947, 0.5789473684210527, 0.3684210526315789, 0.7894736842105263, 0.42105263157894735, 0.7368421052631579, 0.631578947368421, 0.89

# RBM

In [2]:
import rbm

temp = rbm.test_rbm(dataset = featureMat_normed,learning_rate=0.01, training_epochs=15, batch_size=5,n_chains=9,
             n_hidden=9)

print("\n\n")
#print(np.sum(temp, axis=1))

enhanced_feature_sum = []

for i in range(len(np.sum(temp,axis=1))) :
    enhanced_feature_sum.append([np.sum(temp,axis=1)[i],i])
    emptyarray[i][0][2]=np.sum(temp,axis=1)[i]
print("enhanced feature sum",enhanced_feature_sum)
print("\n\n\n")

enhanced_feature_sum.sort(key=lambda x: x[0])
print("enhanced feature sum sorted",enhanced_feature_sum)

length_to_be_extracted = len(enhanced_feature_sum)/2

print("\n\nThe text is : \n\n")
for x in range(len(sentences)):
    print(sentences[x])

print("\n\n\nExtracted sentences : \n\n\n")
extracted_sentences = []
extracted_sentences.append([sentences[0], 0])



for x in range(int(length_to_be_extracted)) :
    if(enhanced_feature_sum[x][1] != 0) :
        extracted_sentences.append([sentences[enhanced_feature_sum[x][1]], enhanced_feature_sum[x][1]])
       

print(extracted_sentences)
extracted_sentences.sort(key=lambda x: x[1])
print("\n\n")
#print(extracted_sentences)


######array method to print summary############
#emparray1 = emptyarray[0]
#emparray2 = emptyarray[1:]
#emparray2 = emparray2[emparray2[:,:,1].argsort()]
#emparray3 = np.concatenate((emparray1,emparray2))
#print(emparray3)



#summary_file = open("gen_summary.txt","w")
summary1=[]
#finalText = ""
#print("\n\n\nExtracted Final Text : \n\n\n")
for i in range(len(extracted_sentences)):
    final_text="\n"+extracted_sentences[i][0]
    final_text_1=extracted_sentences[i][0]
    summary1.append(final_text_1)
    #print(final_text)
#print(summary1)
#print(len(summary1))
    #summary=" ".join(final_text)
#print(summary)
    #finalText = finalText + extracted_sentences[i][0]

    #summary_file.write(final_text)
#summary_file.close()

emparray1 = emptyarray[0]
#print(emparray1)
emparray2 = emptyarray[1:,:,:]

emparray2 = emparray2[np.argsort(emparray2[:,0,2])]
emparray2 = emparray2[::-1]
sh=emparray2.shape[0]
sh=int(sh/2)+1
emparray2=emparray2[:sh]
emparray2 = emparray2[np.argsort(emparray2[:,0,1])]
#print(emparray2)
#emparray3 = np.concatenate((emparray1,emparray2), axis=1)
#print(emparray3)

rbmarray=emparray2[:,:,:2]
rbm_summary = []
for i in range(rbmarray.shape[0]):
    rbm_summary.append(rbmarray[i][0][0])
    
print("RBM summary \n\n",rbm_summary)
#print(rbmarray)

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


(array([[  1.00000000e+00,   1.60000000e+01,   1.50000000e+01,
          2.77862966e-01,   9.28138274e-01,   4.81927711e-02,
          8.94736842e-01,   0.00000000e+00,   4.70588235e-01],
       [  9.68750000e-01,   7.00000000e+00,   6.00000000e+00,
          1.35443063e-01,   9.09440601e-01,   2.40963855e-02,
          4.21052632e-01,   2.50000000e-01,   2.50000000e-01],
       [  9.37500000e-01,   1.30000000e+01,   1.20000000e+01,
          1.47693865e-01,   9.34821429e-01,   3.21285141e-02,
          7.36842105e-01,   2.85714286e-01,   2.14285714e-01],
       [  9.06250000e-01,   1.10000000e+01,   1.00000000e+01,
          1.79935683e-01,   8.85404892e-01,   2.81124498e-02,
          6.31578947e-01,   3.33333333e-01,   2.50000000e-01],
       [  8.75000000e-01,   1.30000000e+01,   1.20000000e+01,
          2.01814378e-01,   9.66575791e-01,   4.41767068e-02,
          7.36842105e-01,   1.42857143e-01,   1.42857143e-01],
       [  8.43750000e-01,   7.00000000e+00,   6.00000000e+00,
  

C:\Users\shashi\Anaconda3\lib\site-packages\theano\tensor\basic.py:2146: UserWarning: theano.tensor.round() changed its default from `half_away_from_zero` to `half_to_even` to have the same default as NumPy. Use the Theano flag `warn.round=False` to disable this warning.
  "theano.tensor.round() changed its default from"


Training epoch 0, cost is  -7.46967147108
Training epoch 1, cost is  -10.7292783214
Training epoch 2, cost is  -8.97224361551
Training epoch 3, cost is  -0.925240257024
Training epoch 4, cost is  -8.63811678658
Training epoch 5, cost is  -6.91554626657
Training epoch 6, cost is  -0.935631699558
Training epoch 7, cost is  -6.81669070628
Training epoch 8, cost is  -5.43565702114
Training epoch 9, cost is  -0.916869436993
Training epoch 10, cost is  -5.55899081283
Training epoch 11, cost is  -4.42139724327
Training epoch 12, cost is  -0.946350036194
Training epoch 13, cost is  -4.69727706827
Training epoch 14, cost is  -3.61769438093
Training took 0.047756 minutes
32
[[ 0.80630484  8.48521234  9.01371921 -1.10412371  1.17440702 -1.96256422
  -1.87438469  0.19512796  1.53275784]
 [-0.50960661  9.52752374  7.69706369  2.37139818  0.90710064 -0.17962896
  -0.34861157 -1.44363343  1.87613039]
 [ 0.73045322 -0.96888054 -1.09554655  1.20526359 -0.10925204 -1.25032814
   0.81698861 -0.86130199 -

# Fuzzy logic

In [3]:
import numpy as np
import matplotlib
%matplotlib inline
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# New Antecedent/Consequent objects hold universe variables and membership
# functions
position1 = ctrl.Antecedent(np.arange(0, 100, 10), 'position1')
cos_similarity = ctrl.Antecedent(np.arange(0, 100, 10), 'cos_similarity')
bitokens = ctrl.Antecedent(np.arange(0, 100, 10), 'bitokens')
tritokens = ctrl.Antecedent(np.arange(0, 100, 10), 'tritokens')
propernoun = ctrl.Antecedent(np.arange(0, 100, 10), 'propernoun')
sentencelength = ctrl.Antecedent(np.arange(0, 100, 10), 'sentencelength')
numtokens = ctrl.Antecedent(np.arange(0, 100, 10), 'numtokens')
keywords = ctrl.Antecedent(np.arange(0, 10, 1), 'keywords')
tf_isf = ctrl.Antecedent(np.arange(0, 100, 10), 'tf_isf')


senten = ctrl.Consequent(np.arange(0, 100, 10), 'senten')

position1.automf(3)
cos_similarity.automf(3)
bitokens.automf(3)
tritokens.automf(3)
propernoun.automf(3)
sentencelength.automf(3)
numtokens.automf(3)
keywords.automf(3)
tf_isf.automf(3)


senten['bad'] = fuzz.trimf(senten.universe, [0, 0, 50])
senten['avg'] = fuzz.trimf(senten.universe, [0, 50, 100])
senten['good'] = fuzz.trimf(senten.universe, [50, 100, 100])

rule1 = ctrl.Rule(position1['good'] & sentencelength['good'] & propernoun['good'] &numtokens['good'], senten['good'])
rule2 = ctrl.Rule(position1['poor'] & sentencelength['poor'] & numtokens['poor'], senten['bad'])
rule3 = ctrl.Rule(propernoun['poor'] & keywords['average'], senten['bad'])
rule4 = ctrl.Rule(cos_similarity['good'], senten['good'])
rule5 = ctrl.Rule(bitokens['good'] & tritokens['good'] & numtokens['average'] | tf_isf['average'], senten['avg'])


sent_ctrl = ctrl.ControlSystem([rule1,rule2,rule3,rule4,rule5])
Sent = ctrl.ControlSystemSimulation(sent_ctrl)
fuzzemptyarr= np.empty((20,1,2), dtype=object)
t2=0
summary2=[]
for s in range(len(sentences)):
    Sent.input['position1'] = int(position[s])
    Sent.input['cos_similarity'] = int(cosine_similarity[s])
    Sent.input['bitokens'] = int(bi_tokens[s])
    Sent.input['tritokens'] = int(tri_tokens[s])
    Sent.input['tf_isf'] = int(tfisfvec[s])
    Sent.input['keywords'] = int(thematic_number[s])
    Sent.input['propernoun'] = int(pnounscore[s])
    Sent.input['sentencelength'] = int(sent_length[s])
    Sent.input['numtokens'] = int(numeric_token[s])
#Sent.input['service'] = 2
    Sent.compute()
    if Sent.output['senten'] > 50:
        summary2.append((sentences[s]))
        fuzzemptyarr[t2][0][0] = sentences[s]
        fuzzemptyarr[t2][0][1] = s
        t2+=1
fuzzarray = np.empty((len(summary2),1,2),dtype=object)
for i in range(len(summary2)):
    fuzzarray[i][0][0] = fuzzemptyarr[i][0][0]
    fuzzarray[i][0][1] = fuzzemptyarr[i][0][1]
    
fuzzarray=fuzzarray[1:]
print("Fuzzy logic summary \n\n",summary2)
#print(len(summary2))
#print(fuzzarray)
    #senten.view(sim=Sent)

Fuzzy logic summary 

 ['Johannesburg: South Africa survived a nail-biting finish to edge out India by five wickets in a rain-curtailed fourth One-Day International (ODI) in Johannesburg on Saturday.', 'The Indian bowlers dominated South African batsmen in the initial stages.', 'That turned out to be the turning point as the match quickly turned on its head.', 'Chinaman Kuldeep Yadav was the most successful among the Indian bowlers with figures of 2/51 while fast bowler Jasprit Bumrah, Pandya and Chahal got a wicket each.', 'Earlier, Dhawan shared a 158-run second-wicket stand with Kohli after opener Rohit Sharma was dismissed early on.', 'He registered his 46th ODI fifty.', 'The stand was broken when Kohli hit pacer Chris Morris straight into the hands of David Miller at short cover, with India at 178/2 in 31.1 overs.', "India's march was halted by lightening with dark clouds hovering over the New Wanderers Stadium.", 'When the match resumed, Dhawan was the first to be dismissed, driv

In [4]:
################common sentences#######################
summarray=np.empty((emparray2.shape), dtype=object) 
t1=0
for i in range(emparray2.shape[0]):
    t = emparray2[i][0][0]
    t2 =emparray2[i][0][1]
    for j in summary2:
        if t == j:
            summarray[t1][0][0] = t
            summarray[t1][0][1] = t2
            t1 += 1
commsentarray=summarray[summarray != np.array(None)]
commonarray = np.empty((int(commsentarray.shape[0]/2),1,2),dtype=object)
for i in range((int(commsentarray.shape[0]/2))):
    commonarray[i][0][0] = summarray[i][0][0]
    commonarray[i][0][1] = summarray[i][0][1]
#print(commonarray)

In [5]:
##############uncommon sentences########################
concarray=np.concatenate((rbmarray,fuzzarray), axis=0)
#print(concarray.shape)
#print(rbmarray.shape)
#print(fuzzarray.shape)

concarray2 = concarray

for i in range(commonarray.shape[0]):
    t1 = commonarray[i][0][0]
    for j in range(concarray.shape[0]):
        t2 = concarray[j][0][0]
        if t1 == t2:
            concarray2[j][0][0] = 0
            concarray2[j][0][1] = 0
            
#print(concarray2)
concarray3=concarray2[concarray2 != 0]
#print(concarray3)
#print(len(concarray3))
uncommonarray= np.empty((int(concarray3.shape[0]/2),1,2), dtype=object)
t1=0
t2=0

for i in range(concarray3.shape[0]):
    if i % 2 == 0:
        uncommonarray[t1][0][0]=concarray3[i]
        t1+=1
    else:
        uncommonarray[t2][0][1]=concarray3[i]
        t2+=1
#print(uncommonarray)
#print(uncommonarray.shape)   
uncommonarray=uncommonarray[np.argsort(uncommonarray[:,0,1])]
uncommonarray=uncommonarray[:int(uncommonarray.shape[0]/2)]
#print(uncommonarray)


######### including first sentence ###########
emparray5=emparray1[0][0]
firstsent=np.empty((1,1,2),dtype=object)
emparray6=emparray1[0][1]
firstsent[0][0][0]=emparray5
firstsent[0][0][1]=emparray6
#print(firstsent)
#print(firstsent.shape)
#print(commonarray.shape)
#print(uncommonarray.shape)

########## final summary ############
final_summ_array=np.concatenate((firstsent,commonarray,uncommonarray))
final_summ_array=final_summ_array[np.argsort(final_summ_array[:,0,1])]
#print(final_summ_array)

final_summary = []
for i in range(final_summ_array.shape[0]):
    final_summary.append(final_summ_array[i][0][0])
print("Final Summary \n\n",final_summary)



Final Summary 

 ['Johannesburg: South Africa survived a nail-biting finish to edge out India by five wickets in a rain-curtailed fourth One-Day International (ODI) in Johannesburg on Saturday.', "Electing to bat first, India rode on opener Shikhar Dhawan's 13th ODI century to post 289/7 in their 50 overs.", 'Dhawan scored 109 runs off 105 balls, and shared a 158-run stand with skipper Virat Kohli who scored 75.', 'The match was interrupted by rain and inclement weather on more than one occasion which saw the target being reduced to 202 runs off 28 overs for South Africa.', 'The Indian bowlers dominated South African batsmen in the initial stages.', 'With their top four batsmen back in the pavilion by the 17th over with just 102 runs on the board, the Proteas seemed to have lost their way and headed for certain defeat.', 'When David Miller was bowled while trying to hit an incoming delivery from leg-spinner Yuzvendra Chahal, the visitors must have thought that the match was as good as 